In [1]:
import torch
import pickle
from utils import extract_full_vector, extract_subset_vector, extract_subset_vector_exp_lip, unflatten_vector, map_subset_to_full_vector, extract_subset_vector_xs_lip
import tqdm
from collections import defaultdict
import numpy as np
import pickle
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import math
import os
import yaml
from datetime import datetime

In [2]:
with open('pkls/dataset_descriptor/live_portrait_descriptor_all_with_mead.pkl', 'rb') as file:
    all_descriptors = pickle.load(file) # frame name is the key and the value is the descriptor

In [3]:
for key in all_descriptors:
    print(key)
    print(all_descriptors[key])
    break

M009/front/happy/level_3/017/frame_0113.jpg
{'c_d_eyes_lst': [array([[0.32350767, 0.3143503 ]], dtype=float32)], 'c_d_lip_lst': [array([[0.28211427]], dtype=float32)], 'driving_template_dct': {'n_frames': 1, 'output_fps': 25, 'motion': [{'scale': array([[1.3642578]], dtype=float32), 'R': array([[[ 0.9988814 ,  0.01929191,  0.0431716 ],
        [-0.01519142,  0.9955155 , -0.09337095],
        [-0.0447793 ,  0.09261066,  0.99469495]]], dtype=float32), 'exp': array([[[ 4.0779114e-03,  1.4381409e-02,  1.6689301e-05],
        [-3.6792755e-03,  2.2468567e-03,  1.1324883e-06],
        [ 3.1051636e-03, -4.1580200e-03,  2.0861626e-06],
        [-2.2184849e-04,  3.9749146e-03,  1.5616417e-05],
        [ 2.3210049e-04,  7.8430176e-03, -1.0013580e-05],
        [-6.9093704e-04, -1.4209747e-03, -3.8146973e-06],
        [ 2.7358532e-05,  5.3584576e-05,  2.3841858e-07],
        [ 1.1243820e-03, -4.9743652e-03, -2.8610229e-06],
        [-1.2218952e-05,  2.4080276e-05, -6.5565109e-07],
        [ 4.52995

In [15]:
# Dictionary to store video frames
video_dict = defaultdict(list)

# Populate the video_dict with frame arrays in order
for key, value in all_descriptors.items():
    parts = key.split('/')
    if key[0] == "M": ## mead dataset
        video_name = "/".join(parts[:-1])
        frame_number = parts[-1].split('.')[0].split("_")[-1]
    else:
        video_name = parts[1] #rawdes
        frame_number = parts[-1].split('.')[0]
    video_dict[video_name].append((frame_number, value))

# Sort video_dict by keys
video_dict = dict(sorted(video_dict.items()))

# Sort each video's frames by frame number
for video_name in video_dict:
    video_dict[video_name].sort(key=lambda x: int(x[0]))

all_vectors_full = []
for video_name, frames in tqdm(video_dict.items(), desc="Extracting vectors for clustering"):
    video_frame_vectors = []
    for frame_number, frame_data in frames:
        vector_full = extract_subset_vector(extract_full_vector(frame_data))
        all_vectors_full.append(vector_full)
all_vectors_full = np.vstack(all_vectors_full)


Extracting vectors for clustering: 100%|██████████| 9282/9282 [03:51<00:00, 40.13it/s]


In [16]:
for key in video_dict:
    parts = key.split('/')
    if key[0] != "M": ## mead dataset
        for k in video_dict[key]:
            print(k[0])
    break
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
# Split into train (90%) and temp (10%)
train_data, temp_data = train_test_split(all_vectors_full, test_size=0.1, random_state=42)

# Split temp into validation (5%) and test (5%)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [5]:
# Convert to PyTorch tensors
train_tensor = torch.tensor(train_data, dtype=torch.float32)
val_tensor = torch.tensor(val_data, dtype=torch.float32)
test_tensor = torch.tensor(test_data, dtype=torch.float32)

In [6]:
# Create TensorDatasets
train_dataset = TensorDataset(train_tensor, train_tensor)  # Input and target are the same
val_dataset = TensorDataset(val_tensor, val_tensor)
test_dataset = TensorDataset(test_tensor, test_tensor)

In [7]:
# Create DataLoaders
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
import torch
import torch.nn as nn

# Function to build a custom MLP (Multi-Layer Perceptron)
def build_mlp(layers, activation_functions):
    modules = []
    for i in range(len(layers) - 1):
        modules.append(nn.Linear(layers[i], layers[i + 1]))
        if activation_functions[i] is not None:
            modules.append(activation_functions[i]())
    return nn.Sequential(*modules)

# Encoder Module
class Encoder(nn.Module):
    def __init__(self, input_dim, layer_dims, activations):
        super(Encoder, self).__init__()
        self.encoder = build_mlp([input_dim] + layer_dims, activations)

    def forward(self, x):
        return self.encoder(x)

# Decoder Module
class Decoder(nn.Module):
    def __init__(self, latent_dim, layer_dims, activations):
        super(Decoder, self).__init__()
        self.decoder = build_mlp([latent_dim] + layer_dims, activations)

    def forward(self, x):
        return self.decoder(x)

# Autoencoder combining Encoder and Decoder
class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(Autoencoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [9]:
# Example configurations
input_dim = 72
encoded_dim = 16
encoder_layers = [input_dim, 512, 256, encoded_dim]  # Number of neurons in each encoder layer
encoder_activations = [nn.ReLU, nn.ReLU, nn.ReLU, nn.ReLU]  # Activation functions for encoder
decoder_layers = [256, 512, input_dim]  # Number of neurons in each decoder layer
decoder_activations = [nn.ReLU, nn.ReLU, None]  # Activation functions for decoder
latent_dim = encoder_layers[-1]  # Latent dimension is the output of the last encoder layer

# Initialize encoder and decoder
encoder = Encoder(input_dim=input_dim, layer_dims=encoder_layers, activations=encoder_activations)
decoder = Decoder(latent_dim=latent_dim, layer_dims=decoder_layers, activations=decoder_activations)

# Combine into autoencoder
autoencoder = Autoencoder(encoder=encoder, decoder=decoder)

In [10]:
# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)

In [11]:
from tqdm import tqdm

# Early stopping parameters
patience = 2  # Number of epochs to wait for improvement
threshold = 1e-4  # Minimum change in validation loss to consider as an improvement
best_val_loss = float('inf')
epochs_no_improve = 0

num_epochs = 50

for epoch in range(num_epochs):
    # Training Phase
    autoencoder.train()
    train_loss = 0
    train_steps = 0
    train_progress = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training", leave=False)

    for inputs, _ in train_progress:
        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

        # Update cumulative average loss
        train_loss += loss.item()
        train_steps += 1
        avg_train_loss = train_loss / train_steps
        train_progress.set_postfix({"Train Loss": avg_train_loss})

    # Validation Phase
    autoencoder.eval()
    val_loss = 0
    val_steps = 0
    val_progress = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation", leave=False)

    with torch.no_grad():
        for inputs, _ in val_progress:
            outputs = autoencoder(inputs)
            loss = criterion(outputs, inputs)

            # Update cumulative average loss
            val_loss += loss.item()
            val_steps += 1
            avg_val_loss = val_loss / val_steps
            val_progress.set_postfix({"Val Loss": avg_val_loss})

    # Print final losses for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    # Early Stopping Check
    if avg_val_loss < best_val_loss - threshold:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"Validation loss did not improve for {epochs_no_improve} consecutive epochs.")

    if epochs_no_improve >= patience:
        print(f"Early stopping triggered at epoch {epoch+1}. Best Val Loss: {best_val_loss:.4f}")
        break


Epoch [1/50] - Train Loss: 0.0172, Val Loss: 0.0012


Epoch [2/50] - Train Loss: 0.0007, Val Loss: 0.0004


Epoch [3/50] - Train Loss: 0.0004, Val Loss: 0.0001


Epoch [4/50] - Train Loss: 0.0003, Val Loss: 0.0001
Validation loss did not improve for 1 consecutive epochs.


Epoch [5/50] - Train Loss: 0.0003, Val Loss: 0.0003
Validation loss did not improve for 2 consecutive epochs.
Early stopping triggered at epoch 5. Best Val Loss: 0.0001


In [12]:
autoencoder.eval()
test_loss = 0
with torch.no_grad():
    for inputs, _ in test_loader:
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader):.10f}")
# Epoch [4/50] - Train Loss: 0.0014, Val Loss: 0.0008 ## 20 ## Test Loss: 0.0012
# Epoch [7/50] - Train Loss: 0.0010, Val Loss: 0.0008 ## 16 ## Test Loss: 0.0008
# Epoch [6/50] - Train Loss: 0.0011, Val Loss: 0.0008 ## 72 ## Test Loss: 0.0008

#latest
#Epoch [6/50] - Train Loss: 0.0003, Val Loss: 0.0002 ## Test loss: 0.0002


Test Loss: 0.0003076470


In [13]:
from torch.utils.data import DataLoader, TensorDataset
import torch
import numpy as np

# Convert numpy array to PyTorch tensor
input_tensor = torch.tensor(all_vectors_full, dtype=torch.float32)

# Create a DataLoader for batching
batch_size = 512
dataset = TensorDataset(input_tensor)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Ensure the encoder is in evaluation mode
autoencoder.encoder.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder.encoder.to(device)

# Container for encoded vectors
encoded_vectors_list = []

# Process the input data in batches
with torch.no_grad():
    for batch in data_loader:
        batch_inputs = batch[0].to(device)  # Get inputs and move to device
        encoded_batch = autoencoder.encoder(batch_inputs)  # Encode the batch
        encoded_vectors_list.append(encoded_batch.cpu())  # Move to CPU and store

# Concatenate all batches into a single tensor
encoded_vectors = torch.cat(encoded_vectors_list, dim=0)

# Convert back to NumPy array (if needed)
encoded_vectors_numpy = encoded_vectors.numpy()

# Print the resulting 20-dimensional vectors
print("Encoded Vectors Shape:", encoded_vectors_numpy.shape)

Encoded Vectors Shape: (1156143, 16)


In [14]:
# import pickle

# # Save the encoded vectors (NumPy array) to a pickle file
# output_file = "auto_encoder_vectors.pkl"

# with open(output_file, "wb") as f:
#     pickle.dump(encoded_vectors_numpy, f)

# print(f"Encoded vectors saved to {output_file}")

def map_subset_to_full_vector_exp_lip(full_vector, subset_output):
    c_lip = subset_output[:1]
    exp = subset_output[1:]
    full_vector[5:6] = c_lip
    full_vector[13:76] = exp
    return full_vector


In [15]:
autoencoder= autoencoder.cuda()

In [16]:
new_dict = dict()
dict_encoder_descriptors = dict()
for key, value in all_descriptors.items():
    full_vector = extract_full_vector(all_descriptors[key])
    subset_vector = extract_subset_vector(full_vector)
    torch_subset_vector = torch.tensor(subset_vector, dtype=torch.float32).cuda()
    encoder_output= autoencoder.encoder(torch_subset_vector)
    decoder_output = autoencoder.decoder(encoder_output).cpu().detach().numpy()

    encoder_output_numpy = autoencoder.encoder(torch_subset_vector).cpu().detach().numpy()
    new_dict[key] = encoder_output_numpy

    full_vector = map_subset_to_full_vector(full_vector=full_vector, subset_output=decoder_output)
    dict_encoder_descriptors[key] = unflatten_vector(full_vector)

In [17]:
import pickle
# Save the encoded vectors (NumPy array) to a pickle file
output_file = "pkls/auto_encoder_output/encoded_live_portrait_descriptor_all_with_mead.pkl"

with open(output_file, "wb") as f:
    pickle.dump(new_dict, f)

print(f"Encoded vectors saved to {output_file}")

Encoded vectors saved to pkls/auto_encoder_output/encoded_live_portrait_descriptor_all_with_mead.pkl


In [18]:
import pickle
# Save the encoded vectors (NumPy array) to a pickle file
output_file = "pkls/auto_encoder_output/autodecoded_descriptors_live_portrait_descriptor_all_with_mead.pkl"

with open(output_file, "wb") as f:
    pickle.dump(dict_encoder_descriptors, f)

print(f"Encoded vectors saved to {output_file}")

Encoded vectors saved to pkls/auto_encoder_output/autodecoded_descriptors_live_portrait_descriptor_all_with_mead.pkl


In [19]:
import torch

# File path to save the encoder
encoder_save_path = "trained_models/encoder_16_with_mead_5_148.pth"

# Save the encoder's state dictionary
torch.save(autoencoder.encoder.state_dict(), encoder_save_path)

print(f"Encoder saved successfully to {encoder_save_path}")


Encoder saved successfully to trained_models/encoder_16_with_mead_5_148.pth
